# Colab Setup
These steps are necessary to copy the github repo into your google drive folder. 

Check out [Vortana's tutorial](https://towardsdatascience.com/google-drive-google-colab-github-dont-just-read-do-it-5554d5824228) for the colab/github/drive integration.
Most of the code is taken from his work.

Before starting you'll want to allocate some GPUs.

   - Navigate to Edit→Notebook Settings
   - Select GPU from the Hardware Accelerator drop-down

In [1]:
# Mount Google Drive
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
print(ROOT)  
drive.mount(ROOT, force_remount=True)

/content/drive
Mounted at /content/drive


In [2]:
# Clone github repository setup
# import join used to join ROOT path and MY_GOOGLE_DRIVE_PATH
from os.path import join  

# path to your project on Google Drive
MY_GOOGLE_DRIVE_PATH = "MyDrive/ColabNotebooks"
# replace with your Github username 
GIT_USERNAME = "nicofirst1" 
# definitely replace with your
GIT_TOKEN = "{YOUR TOKEN}"  
# Replace with your github repository 
GIT_REPOSITORY = "MAMBRL" 

PROJECT_PATH = join(ROOT, MY_GOOGLE_DRIVE_PATH)

# It's good to print out the value if you are not sure 
print("PROJECT_PATH: ", PROJECT_PATH)   

# In case we haven't created the folder already; we will create a folder in the project path 
!mkdir "{PROJECT_PATH}"    

GIT_PATH = "https://" + GIT_TOKEN + "@github.com/" + GIT_USERNAME + "/" + GIT_REPOSITORY + ".git"
print("GIT_PATH: ", GIT_PATH)

PROJECT_PATH:  /content/drive/MyDrive/ColabNotebooks
mkdir: cannot create directory ‘/content/drive/MyDrive/ColabNotebooks’: File exists
GIT_PATH:  https://{YOUR TOKEN}@github.com/nicofirst1/MAMBRL.git


In [5]:
# copy git repo in project path
%cd "{PROJECT_PATH}"
%rm -rf "{GIT_REPOSITORY}"
!git clone "{GIT_PATH}"
%cd "{GIT_REPOSITORY}"

/content/drive/MyDrive/ColabNotebooks
Cloning into 'MAMBRL'...
remote: Enumerating objects: 157, done.
remote: Counting objects: 100% (157/157), done.
remote: Compressing objects: 100% (110/110), done.
remote: Total 157 (delta 74), reused 126 (delta 43), pack-reused 0
Receiving objects: 100% (157/157), 46.64 KiB | 901.00 KiB/s, done.
Resolving deltas: 100% (74/74), done.
/content/drive/MyDrive/ColabNotebooks/MAMBRL


# Python setup and training



In [ ]:
# install requirements
!pip install -r requirements.txt


In [49]:
#import python files
import importlib

training = importlib.import_module('src.model.training')
utils = importlib.import_module('src.utils.utils')
params = importlib.import_module('src.utils.Params')
NavEnv = importlib.import_module('env.NavEnv')


exec(open("./src/model/training.py").read(), training.__dict__)
exec(open("./src/utils/utils.py").read(), utils.__dict__)
exec(open("./src/utils/Params.py").read(), params.__dict__)
exec(open("./env/NavEnv.py").read(), NavEnv.__dict__)




In [50]:
# Setup configurations

import ray
from ray.tune.logger import pretty_print
from ray.tune import register_env
from ray.tune.integration.wandb import WandbLoggerCallback

params = params.Params()
ray.init(local_mode=False, num_gpus=0, num_cpus=16,ignore_reinit_error=True, log_to_driver=False)

# Get configs
env_config =  utils.get_env_configs(params)
policy_configs =  utils.get_policy_configs(params)
model_configs =  utils.get_model_configs(params)
callbacks=[]


configs = {
    "env": params.env_name,
    "env_config": env_config,
    "framework": params.framework,
    "num_workers": 1,
    "batch_mode": "complete_episodes",
    #"train_batch_size": 400,
    "rollout_fragment_length": 50,

    # PPO parameter
    "lr": 3e-4,
    "lambda": .95,
    "gamma": .998,
    "entropy_coeff": 0.01,
    "clip_param": 0.2,
    "use_critic": True,
    "use_gae": True,
    "grad_clip": 5,
    "num_sgd_iter": 10,

    # Model
    "model": model_configs,

    # Multiagent
    "multiagent": policy_configs
}

# register the navigation environment
register_env(params.env_name,
             lambda config: NavEnv.get_env(config))

# get wandb logger
WANDB_API_KEY = "{YOUR KEY}"

if len(WANDB_API_KEY)>15:
    wand_logger = WandbLoggerCallback(project=params.env_name, api_key=WANDB_API_KEY, log_config=True)
    callbacks.append(wand_logger)
    print("WanDB logger initialized!")


2021-10-19 16:36:26,658	INFO worker.py:836 -- Calling ray.init() again after it has already been called.
/usr/local/lib/python3.7/dist-packages/pettingzoo/utils/wrappers/base.py:44: UserWarning: The `observation_spaces` dictionary is deprecated. Use the `observation_space` function instead.
  warnings.warn("The `observation_spaces` dictionary is deprecated. Use the `observation_space` function instead.")
/usr/local/lib/python3.7/dist-packages/pettingzoo/utils/wrappers/base.py:52: UserWarning: The `action_spaces` dictionary is deprecated. Use the `action_space` function instead.
  warnings.warn("The `action_spaces` dictionary is deprecated. Use the `action_space` function instead.")


In [52]:
# Start training

checkpoint_path, analysis = training.tune_train(params, configs, callbacks)
pretty_print(analysis)

(pid=1493) /usr/local/lib/python3.7/dist-packages/pettingzoo/utils/wrappers/base.py:44: UserWarning: The `observation_spaces` dictionary is deprecated. Use the `observation_space` function instead.
(pid=1493)   warnings.warn("The `observation_spaces` dictionary is deprecated. Use the `observation_space` function instead.")
(pid=1493) /usr/local/lib/python3.7/dist-packages/pettingzoo/utils/wrappers/base.py:52: UserWarning: The `action_spaces` dictionary is deprecated. Use the `action_space` function instead.
(pid=1493)   warnings.warn("The `action_spaces` dictionary is deprecated. Use the `action_space` function instead.")
2021-10-19 16:36:48,577	WARNING util.py:55 -- Install gputil for GPU system monitoring.
/usr/local/lib/python3.7/dist-packages/pettingzoo/utils/wrappers/base.py:44: UserWarning: The `observation_spaces` dictionary is deprecated. Use the `observation_space` function instead.
  warnings.warn("The `observation_spaces` dictionary is deprecated. Use the `observation_space`

agent_timesteps_total: 4000
custom_metrics: {}
date: 2021-10-19_16-37-08
done: false
episode_len_mean: 200.0
episode_media: {}
episode_reward_max: -10585.600000000002
episode_reward_mean: -107139.7
episode_reward_min: -196279.99999999997
episodes_this_iter: 20
episodes_total: 20
experiment_id: c5f06b844a024e15ac18b9ceb72bc678
hostname: d161497c9b66
info:
  learner:
    agent_0:
      custom_metrics: {}
      learner_stats:
        cur_kl_coeff: 0.20000000298023224
        cur_lr: 0.0003000000142492354
        entropy: 1.604877233505249
        entropy_coeff: 0.009999999776482582
        kl: 0.004541185684502125
        policy_loss: -0.006357148289680481
        total_loss: 101827016.0
        vf_explained_var: -1.9830093151540495e-05
        vf_loss: 101827016.0
      model: {}
    agent_1:
      custom_metrics: {}
      learner_stats:
        cur_kl_coeff: 0.20000000298023224
        cur_lr: 0.0003000000142492354
        entropy: 1.6036792993545532
        entropy_coeff: 0.00999999977

KeyboardInterrupt: ignored